In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch.optim as optim

from efficientnet_pytorch import EfficientNet
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim import lr_scheduler

from hw_grapheme.train import generate_stratified_k_fold_index, train_model
from hw_grapheme.utils import load_model_weight
from hw_grapheme.data_pipeline import create_dataloaders, load_data
from hw_grapheme.model import EfficientNet_0
from hw_grapheme.loss_func import Loss_combine

from torchtools.optim import RangerLars, RAdam
# from one_cycle import OneCycleLR
from torch.optim import Optimizer

# from warmup_scheduler import GradualWarmupScheduler

from apex import amp

To use this log_lamb_rs, please run 'pip install tensorboardx'. Also you must have Tensorboard running to see results


In [3]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
#     mean = np.array([0.485, 0.456, 0.406])
#     std = np.array([0.229, 0.224, 0.225])
#     inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [4]:
# load data 
pickle_paths = [
    "../data/processed_data/size_224/train_data_0.pickle",
    "../data/processed_data/size_224/train_data_1.pickle",
    "../data/processed_data/size_224/train_data_2.pickle",
    "../data/processed_data/size_224/train_data_3.pickle",
]

image_data, name_data, label_data = load_data(pickle_paths)

Load data done, shape: (200840, 224, 224), (200840,), (200840, 3)


In [5]:
batch_size = 200
num_workers = 6
pin_memory = True
n_epoch = 10

n_splits = 5
random_seed = 2020

mixed_precision = False

train_idx_list, test_idx_list = generate_stratified_k_fold_index(
    image_data, label_data, n_splits, random_seed
)

# create loss function
# criterion = nn.CrossEntropyLoss()
criterion = Loss_combine()

# for discriminative lr
# my_list = ['module._fc.weight', 'module._fc.bias']
# params = list(filter(lambda kv: kv[0] in my_list, eff_b0.named_parameters()))
# base_params = list(filter(lambda kv: kv[0] not in my_list, eff_b0.named_parameters()))
# params = [kv[1] for kv in params]
# base_params = [kv[1] for kv in base_params]

# create data_transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.ToPILImage(),
        # transforms.RandomAffine(degrees=10, scale=(1.0, 1.15)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        # transforms.Normalize([0.0692], [0.2051]),
        # transforms.ToPILImage(),
    ]),
    'val': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        # transforms.Normalize([0.0692], [0.2051])
    ]),
}

StratifiedKFold(n_splits=5, random_state=2020, shuffle=True)


In [6]:
pretrain = False

for i, (train_idx, valid_idx) in enumerate(zip(train_idx_list, test_idx_list)):
    
    print(f"Training fold {i}")
    
    # create model 
    eff_b0 = EfficientNet_0()
        
    # create optimizer
    optimizer_ft = optim.Adam(eff_b0.parameters(), weight_decay=1e-5)

    if mixed_precision:
        opt_level = "O1"
        eff_b0.to("cuda")
        eff_b0, optimizer_ft = amp.initialize(eff_b0, optimizer_ft, opt_level=opt_level)
        eff_b0 = nn.DataParallel(eff_b0)
    else:
        eff_b0.to("cuda")
        eff_b0 = nn.DataParallel(eff_b0)
        
    # create data loader
    data_loaders = create_dataloaders(
        image_data, name_data, label_data, train_idx, valid_idx, 
        data_transforms, batch_size, num_workers, pin_memory
    )
         
    # create lr scheduler
    # exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(
    #     optimizer_ft, factor=0.5, patience=5,
    # )
#     cos_lr_scheduler = lr_scheduler.CosineAnnealingLR(
#         optimizer_ft, T_max=n_epoch,
#     )
#     # one_cycle_lr_scheduler = OneCycleLR(
#     #     optimizer_ft, max_lr=0.01, steps_per_epoch=len(data_loaders["train"]), epochs=n_epoch
#     # )   
    
#     scheduler_warmup = GradualWarmupScheduler(
#         optimizer_ft, multiplier=1, total_epoch=10, after_scheduler=cos_lr_scheduler
#     )

    
    callbacks = {}

    callbacks = train_model(
        eff_b0, criterion, optimizer_ft, data_loaders,
        mixed_precision, callbacks, num_epochs=n_epoch,
        epoch_scheduler=None, 
        save_dir=f"../model_weights/eff_0_baseline_with_mixed_precision/fold_{i}"
    )


Training fold 0
Creating train dataloader...
Creating test dataloader...
Epoch 0/9
----------



Train Loss: 1.2234, root_acc: 0.6177, vowel_acc: 0.7062, consonant_acc: 0.8373, combined_acc: 0.6947



Val Loss: 0.6850, root_acc: 0.7687, vowel_acc: 0.8945, consonant_acc: 0.9001, combined_acc: 0.8330
In epoch 0, highest val accuracy increases from 0.0 to 0.8330200657239594.
In epoch 0, lowest val loss decreases from 999 to 0.6849625125939902.

Epoch 1/9
----------



Train Loss: 0.3571, root_acc: 0.8735, vowel_acc: 0.9286, consonant_acc: 0.9492, combined_acc: 0.9062



Val Loss: 0.4162, root_acc: 0.8489, vowel_acc: 0.9372, consonant_acc: 0.9512, combined_acc: 0.8966
In epoch 1, highest val accuracy increases from 0.8330200657239594 to 0.8965718980282812.
In epoch 1, lowest val loss decreases from 0.6849625125939902 to 0.4161845112055855.

Epoch 2/9
----------


...


Train Loss: 0.1049, root_acc: 0.9620, vowel_acc: 0.9734, consonant_acc: 0.9795, combined_acc: 0.9692



Val Loss: 0.3230, root_acc: 0.8980, vowel_acc: 0.9613, consonant_acc: 0.9652, combined_acc: 0.9306
In epoch 8, highest val accuracy increases from 0.929645488946425 to 0.9306288587930691.

Epoch 9/9
----------



Train Loss: 0.0952, root_acc: 0.9656, vowel_acc: 0.9750, consonant_acc: 0.9805, combined_acc: 0.9717



Val Loss: 0.2947, root_acc: 0.9098, vowel_acc: 0.9636, consonant_acc: 0.9688, combined_acc: 0.9380
In epoch 9, highest val accuracy increases from 0.9306288587930691 to 0.9380165803624776.
In epoch 9, lowest val loss decreases from 0.3166044826373188 to 0.2946669153781603.

Training complete in 48m 52s
Best Combnied Acc: 0.938017
Training fold 1
Creating train dataloader...
Creating test dataloader...
Epoch 0/9
----------


KeyboardInterrupt: 

In [ ]:
configs = {
    "model": "efficient 0",
    "pretrain": pretrain,
    "head_info": "1 fc",
    "input_size": "224X224",
    "optimizer": "adam",
    "n_fold": n_splits,
    "split_seed": random_seed,
    "batch_size": batch_size,
    "epoch": n_epoch,
    "mixed_precision": mixed_precision
}

In [ ]:
# Get a batch of training data for demo

# visual_loader = DataLoader(
#     train_dataset, batch_size=4,
#     num_workers=num_workers, pin_memory=True,
# )

# inputs, a,b,c = next(iter(visual_loader))

# # Make a grid from batch
# out = torchvision.utils.make_grid(inputs)

# imshow(out)